In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandarallel import pandarallel
from astropy.io import fits
pandarallel.initialize(progress_bar=True)
import random
from numpy.random import default_rng
import numpy as np
rng = default_rng(42)
import shutil
import glob
from scipy import ndimage
import os


Bad key text.latex.preview in file /opt/anaconda/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /opt/anaconda/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /opt/anaconda/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.')
You probably

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
def abrir_fits(ruta):
    hdu_list = fits.open(ruta,memmap=False)
    image_data = hdu_list[0].data
    hdu_list.close()
    return image_data

In [4]:
#### INCLINATED GALAXIES ####
au_ss_rot_incl =  pd.read_csv('list_incl_rand.txt',header=None)
path_fits_incl_rand = "/data/processing/avera/proyecto_deep/fits_inclinated_rand/"
fits_files_incl_rand = glob.glob( os.path.join(path_fits_incl_rand, "*"))
detec_rand = pd.read_csv('Detecciones_inclinated_rand.csv')
datos = pd.DataFrame({"direccion":fits_files_incl_rand})
datos['label']=0
datos['snap']=-1
datos['sb']=0
datos['incl']=-1
datos['rot']=-1

count=0

for n in range(len(detec_rand)):
    au = detec_rand['AU'][n]
    ss = detec_rand['ss'][n]
    incl = detec_rand['incl'][n]
    rot = detec_rand['rot'][n]
    
    for m in range(10):
        sup = 24+m
        mask = datos['direccion'] == f'/data/processing/avera/proyecto_deep/fits_inclinated_rand/AU{au}_snap{ss}_rot{rot}_incl{incl}_SB{sup}.fits'
    
        if detec_rand[str(sup)][count]==1:
            datos.loc[mask,'label'] = 1
        
        datos.loc[mask,'AU']=au
        datos.loc[mask,'sb']=sup
        datos.loc[mask,'incl']=rot
        datos.loc[mask,'snap']=ss
        datos.loc[mask,'rot']=incl
        
    count +=1

In [5]:
datos['name']=datos.label.apply(str)+"_"+datos.direccion.str.split('/').str[6].str.split("_").str[0]+"_"+datos.direccion.str.split('/').str[6].str.split("_").str[1]+"_rot"+datos.direccion.str.split('/').str[6].str.split("_").str[3].str[4:]+"_incl"+datos.direccion.str.split('/').str[6].str.split("_").str[2].str[3:]+"_"+datos.direccion.str.split('/').str[6].str.split("_").str[4]

In [6]:
datos["image"] = datos.parallel_apply(lambda row : abrir_fits(row["direccion"]), axis=1)

In [7]:
datos["label"] = datos.label.astype("int")

In [11]:
np.savetxt('indicesge26.dat',np.array(datos[datos['sb']>=26].index))

In [8]:
x_train = np.empty((len(datos[datos['snap']>105]),300, 300,3))

for i,ima in enumerate(datos["image"][datos['snap']>105]):
    img2 = np.zeros( ( np.array(ima).shape[0], np.array(ima).shape[1], 3 ) )
    img2[:,:,0] = ima # same value in each channel
    img2[:,:,1] = ima
    img2[:,:,2] = ima
    x_train[i] = img2


In [10]:
import h5py
with h5py.File("/data/processing/avera/proyecto_deep/inclinated_rand2.h5", 'w') as hdf:
    # Guardar set de entrenamiento
    hdf.create_dataset('x_test', data=x_train)
    hdf.create_dataset('test_labels', data=datos[datos['snap']>105].label)
    hdf.create_dataset('test_sb', data=datos[datos['snap']>105].sb)
    
    # Guardar set de validación
    #hdf.create_dataset('x_val', data=x_val)
    #hdf.create_dataset('val_labels', data=val.label)
    
    # Guardar set de testeo
    #hdf.create_dataset('x_test', data=x_test)
    #hdf.create_dataset('test_labels', data=test.label)
